# Bible by Theme Interpretation

## Bible by Theme Interpretation: v1

In [ ]:
import json
import os
import pandas as pd
from utils.os_utils import find_repo_root

# Setting directory
repo_root = find_repo_root()
os.chdir(repo_root)

# Loading Fragments of the Bible
with open('bible/data/bible_by_theme_preprocessed.json') as f:
    fragments = json.load(f)

# Loading json in df
fragments_df = pd.DataFrame(fragments)

In [ ]:
import openai
import os

client = openai.OpenAI()

def get_completion_4o(prompt, model="gpt-4o"):
    messages=[
       {"role": "user", "content": prompt}
       ]    
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

In [ ]:
# Prompt Instruction: v1
instructions = """
# Rol #
Tu eres el GPT: Catholic Bible Guide by Fr. Abraham Mutholath.
Tu rol será el de un sacerdote de una iglesia Católica. Se te dará distintos fragmentos de la Biblia. Tu tarea será leer los fragmentos y realizar las tareas que se detallen en la siguiente sección.
# Fin Rol #

# Tareas #
- Explicar la Biblia (explicacion): si el fragmento de la Biblia no está escrito en un lenguaje antiguo, es ambiguo o no está claro lo que quiere decir debes hacer una explicación breve y concisa. Solo explica lo que no es tan evidente y esté en un lenguaje antiguo o ambiguo.
- Identificar Area de la vida (areas): de cada fragmento debes identificar el área de la vida de la que se habla. Las áreas de la vida pueden ser: Relaciones familiares/pareja, Amistad, Salud física, Salud mental/emocional, Desarrollo/crecimiento personal, Misión de vida, Dinero, Trabajo, Diversión/Hobbies, Otros.

Para cada fragmento menciona entre 1 y 3 áreas que sean las más importantes.
# fin Tareas #

# Estructura input #
"id_libro|fragmento"
# Fin estructura input #

# Estructura Output #
El output debe ser un texto plano separado por | sin "header" ni "footer"
id_libro|explicacion|areas(separado por comas)
# Fin Estructura Output #
"""

In [ ]:
list_response = []

for _, verse in fragments_df.iterrows():
    input_to_interpret = f"{verse['pasaje']}|{verse['texto']}"
    prompt = f"""
                {instructions}

                # Input # 
                {input_to_interpret}
                # Fin Input #
                """
    response_4o = get_completion_4o(prompt)
    list_response.append(response_4o)

In [ ]:
list_purpose = fragments_df['area_vida'].tolist()
list_text = fragments_df['texto'].tolist()
list_passage = fragments_df['pasaje'].tolist()

In [ ]:
def flatten_text(text):
    flattened_text = text.replace('\n', ' ')
    flattened_text = ' '.join(flattened_text.split())
    return flattened_text

list_response2 = [flatten_text(response) for response in list_response]

In [ ]:
# Splitting ChatGPT response into passage, interpretatino and areas
list_passage_2 = []
list_interpretation = []
list_areas = []
list_id = []

i = 0
for item in list_response2:
    passage, interpretation = item.split('|')
    i=i+1
    id = "vec"+str(i)
    list_id.append(id)
    list_passage_2.append(passage)
    list_interpretation.append(interpretation)
    #list_areas.append(area)

In [ ]:
# Splitting the list_areas to store them as a list and not as a joined string
# Also trim the white spaces
list_areas_split = [[area.strip() for area in areas.split(',')] for areas in list_areas]
print(list_areas_split)

In [ ]:
# Creating df with results
bible_by_theme_interpreted = pd.DataFrame({"id": list_id,
                                           "pasaje": list_passage,
                                           "texto": list_text,
                                           "interpretación": list_interpretation,
                                           "temas": list_purpose,
                                           "área_vida": list_areas_split})

# Convert DataFrame to a dictionary
bible_by_theme_interpreted_json = bible_by_theme_interpreted.to_dict(orient='records')

# Save the list as a JSON file
with open('bible/data/bible_by_theme_int_v1.json', 'w', encoding='utf-8') as json_file:
    json.dump(bible_by_theme_interpreted_json, json_file, ensure_ascii=False, indent=4)

## Bible by Theme Interpretation: v2

In [7]:
import json
import os
import pandas as pd
from utils.os_utils import find_repo_root

# Setting directory
repo_root = find_repo_root()
os.chdir(repo_root)

# Loading Fragments of the Bible
with open('bible/data/bible_by_theme_preprocessed.json') as f:
    fragments = json.load(f)

# Loading json in df
fragments_df = pd.DataFrame(fragments)

In [3]:
import openai
import os

client = openai.OpenAI()

def get_completion_4o(prompt, model="gpt-4o"):
    messages=[
       {"role": "system", "content": "Tú eres un sacerdote de una iglesia católica"},
       {"role": "user", "content": prompt}
       ]    
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

In [4]:
# Prompt v2
instructions = """
# Rol #
Tu eres el GPT: Catholic Bible Guide by Fr. Abraham Mutholath.
Tu rol será el de un sacerdote de una iglesia Católica. Se te dará distintos fragmentos de la Biblia. Tu tarea será leer los fragmentos y realizar las tareas que se detallen en la siguiente sección.
# Fin Rol #

# Tarea #
explicacion: Explica brevemente el significado espiritual del pasaje bíblico. Relaciona el pasaje con una situación específica en la vida diaria moderna. Incluye detalles que reflejen preocupaciones o desafíos contemporáneos, como trabajo, relaciones, salud, finanzas, etc. Explica como la enseñanza del pasaje puede aplicarse o brindar consuelo en la situación moderna descrita, conectando explícitamente las enseñanzas bíblicas con los desafíos actuales.
# fin Tareas #

# Estructura input #
id_libro|fragmento
# Fin estructura input #

# Estructura Output #
El output debe ser un texto plano separado por | sin "header" ni "footer"
id_libro|explicacion
# Fin Estructura Output #
"""

In [6]:
list_response = []

for _, verse in fragments_df.iterrows():
    input_to_interpret = f"{verse['pasaje']}|{verse['texto']}"
    prompt = f"""
                {instructions}

                # Input # 
                {input_to_interpret}
                # Fin Input #
                """
    response_4o = get_completion_4o(prompt)
    list_response.append(response_4o)

In [10]:
list_purpose = fragments_df['area_vida'].tolist()
list_text = fragments_df['texto'].tolist()
list_passage = fragments_df['pasaje'].tolist()

In [11]:
def flatten_text(text):
    flattened_text = text.replace('\n', ' ')
    flattened_text = ' '.join(flattened_text.split())
    return flattened_text

list_response2 = [flatten_text(response) for response in list_response]

#### Merging all the info

In [13]:
# Splitting ChatGPT response into passage, interpretatino and areas
list_passage_2 = []
list_interpretation = []
list_areas = []
list_id = []

i = 0
for item in list_response2:
    passage, interpretation = item.split('|')
    i=i+1
    id = "vec"+str(i)
    list_id.append(id)
    list_passage_2.append(passage)
    list_interpretation.append(interpretation)

In [14]:
# Creating df with the results
bible_by_theme_interpreted = pd.DataFrame({"id": list_id,
                                           "pasaje": list_passage,
                                           "texto": list_text,
                                           "interpretación": list_interpretation,
                                           "temas": list_purpose})

# Convert DataFrame to a dictionary
bible_by_theme_interpreted_json = bible_by_theme_interpreted.to_dict(orient='records')

# Save the list as a JSON file
with open('bible/data/bible_by_theme_int_v2.json', 'w', encoding='utf-8') as json_file:
    json.dump(bible_by_theme_interpreted_json, json_file, ensure_ascii=False, indent=4)

## Catholic Bible Interpretation: v3

In [ ]:
# Importing library and setting working directory
import json
import pandas as pd
import os
from utils.os_utils import find_repo_root

pd.set_option('display.max_colwidth', None)
pd.options.mode.chained_assignment = None  # default='warn'

# Setting working directory
repo_root = find_repo_root()
os.chdir(repo_root)

In [ ]:
# Loading the data
with open('bible/data/catholic_bible.json') as f:
    data = json.load(f)

# Converting json into df
bible_data = pd.DataFrame(data)
bible_data.head()

In [ ]:
bible_data.drop(columns=['texto'], inplace=True)
bible_data.rename(columns={'texto_cat':'texto'}, inplace=True)
bible_data.head()

In [ ]:
import openai
import os
import time

client = openai.OpenAI()

def get_completion_4o(prompt, model="gpt-4o"):
    messages=[
       {"role": "user", "content": prompt}
       ]    
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=1,
        top_p=1.0
    )
    return response
    #return response.choices[0].message.content

In [ ]:
# Prompt v3
instructions1 = """
# Rol #
Eres un experto en interpretación bíblica con un profundo entendimiento de la vida moderna. Tu tarea es interpretar pasajes de la Biblia de una manera que sea relevante y aplicable a situaciones cotidianas actuales.
# Fin Rol #

# Tarea #
Para cada pasaje bíblico proporcionado:
1. Lee cuidadosamente el texto.
2. Identifica el mensaje principal o la enseñanza clave.
3. Crea una situación moderna que refleje el tema central del pasaje.
4. Describe esa situación moderna y explica como el mensaje del pasaje se aplica.
- **No incluyas etiquetas como '# Output #' en tu respuesta.**
- **No incluyas saltos de línea en tu respuesta.**
- **No incluyas etiquetas como "Situación moderna:" o "Explicación:"**

Tu Respuesta debe tener lo siguiente:
Crea una situación moderna que refleje el tema central del pasaje. Describe esa situación moderna y explica como el mensaje del pasaje se aplica.
# fin Tareas #

# Estructura input #
id_libro|fragmento
# Fin estructura input #
"""

instructions2 = """
# Estructura Output #
id_libro|explicacion
# Fin Estructura Output #
"""

In [ ]:
# Processing the Bible data

# Start timer
start_time = time.time()

list_response = []
# Lists to calculate the tokens used in the processing
list_token_input = []
list_token_output = []

for _, verse in bible_data.iterrows():
    input_to_interpret = f"{verse['pasaje']}|{verse['texto']}"
    prompt = f"""
                {instructions1}
                # Input # 
                {input_to_interpret}
                # Fin Input #
                {instructions2}
                """
    response = get_completion_4o(prompt)
    response_4o = response.choices[0].message.content
    token_input = response.usage.prompt_tokens
    token_output = response.usage.completion_tokens
    list_response.append(response_4o)
    list_token_input.append(token_input)
    list_token_output.append(token_output)

# End the timer
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken to process: {elapsed_time:.2f} seconds")

In [ ]:
# Cost of the API call
sum(list_token_input)*5/1000000 + sum(list_token_output)*15/1000000

In [ ]:
# Cleaining the output
def clean_output(text_list):
    cleaned_list = []
    for text in text_list:
        # Remove leading/trailing whitespace and newline characters
        cleaned_text = text.strip()
        # Replace multiple spaces with a single space
        cleaned_text = ' '.join(cleaned_text.split())
        # Append cleaned text to the list
        cleaned_list.append(cleaned_text)
    return cleaned_list

cleaned_output = clean_output(list_response)

In [ ]:
# Splitting the output into columns
def split_into_columns(text_list):
    list_passage = []
    list_interpretation = []
    
    for text in text_list:
        # Split each text into two parts: the scripture reference and the description
        passage, interpretation = text.split('|', 1)
        list_passage.append(passage.strip())
        list_interpretation.append(interpretation.strip())
    
    import pandas as pd
    df = pd.DataFrame({
        'pasaje_out': list_passage,
        'interpretacion': list_interpretation
    })

    return df

output = split_into_columns(cleaned_output)

In [ ]:
bible_data['pasaje_out'] = output['pasaje_out']
bible_data['interpretacion'] = output['interpretacion']
bible_data.drop(columns=['pasaje_out'], inplace=True)

In [ ]:
# Convert DataFrame to a dictionary
bible_data_json = bible_data.to_dict(orient='records')

# Save the list as a JSON file
with open('bible/data/cat_bible_int_openai.json', 'w', encoding='utf-8') as json_file:
    json.dump(bible_data_json, json_file, ensure_ascii=False, indent=4)